# Accident locations in Tel Aviv between 2018 and 2023
## Main Chapters:
### - Examining and preprocessing the data for accident locations in Tel Aviv for the years 2018(jan) - 2023(july)
### - Focusing and creating tables of accidents by street, and intersection
### - Creating tables summing the injuries per involved vehicle and injury severity



***Using the Tel Aviv data from for only injuries of any severity, 2018/Jan - 2023/Jul (none injured excluded out)***


By Anyway

## Importing libraries

In [1]:
import pandas as pd
import numpy as np

## Reading the data

### Reading csv file created bt Anyway's query
The query includes:
- Only injuries
- From 2018(jan) - 2023(jul -- so far)
- In Tel Aviv cidty code (5000)

In [2]:
tlv_accidents = pd.read_csv('tlv_injured_for_candidates_2018-2023_jul_chez_mod.csv')

#sorting the columns, and removing  2 uneeded columns
tlv_accidents = tlv_accidents[[
        'provider_and_id', #unique accident id code (provider + non unique id create a unique accident id)
#         'road_type_hebrew', #not needed as when column `street2` "isnull" then the road type is "עירונית לא בצומת"
        'street1_hebrew', #the main street where accident happened, always has values
        'street2_hebrew', #when accident happened in an intersection, shows street2 that intersects with street1
        'location_accuracy_hebrew', #shows how accurate the location is
        'accident_type_hebrew', #what or how it happened (fire, cotrol loss, frontal \ side crash, animals, etc.)
        'accident_severity_hebrew', #if any injured person in the accident is light, serious, lethal
        'injury_severity_hebrew', #if the injury of the specific person light, serious, lethal
        'injured_type_modified_by_anyway', #modified column that combines the following two columns
        'injured_type_hebrew', #who was injured - pedestrian, driver, passenger
        'involve_vehicle_type_hebrew', #what vehicle is involved
#         'vehicle_vehicle_type_hebrew', #exactly the same as previous column - what vehicle is involved
        'involved_type_hebrew', # is the person a driver or not?
        'injured_position_hebrew', #what is the position of the injred person (unknown, pedestrian, sat, stand etc.)
        'sex_hebrew', #gender
        'age_group_hebrew', #age group of 5 years (25-29, 30-34, etc.)
        'accident_day', #day of the month (1-31)
        'accident_month', #month number (1-12)
        'accident_year', #year (2018-2023)
        'accident_hour_raw_hebrew', #time of accident in 15 minutes intervals (2:00-2:14, 2:15-2:29, etc.)
        'day_in_week_hebrew' #day of the week (sunday - saturday)               
        ]]

In [3]:
tlv_accidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25473 entries, 0 to 25472
Data columns (total 19 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   provider_and_id                  25473 non-null  int64 
 1   street1_hebrew                   24255 non-null  object
 2   street2_hebrew                   6620 non-null   object
 3   location_accuracy_hebrew         25473 non-null  object
 4   accident_type_hebrew             25473 non-null  object
 5   accident_severity_hebrew         25473 non-null  object
 6   injury_severity_hebrew           25473 non-null  object
 7   injured_type_modified_by_anyway  25473 non-null  object
 8   injured_type_hebrew              25473 non-null  object
 9   involve_vehicle_type_hebrew      22374 non-null  object
 10  involved_type_hebrew             25473 non-null  object
 11  injured_position_hebrew          25473 non-null  object
 12  sex_hebrew                      

### Columns
**The data contains the following columns:**
1. `provider_and_id` — unique accident id code (provider + non unique id create a unique accident id).
2. `street1_hebrew` — the main street where accident happened, always has values.
3. `street2_hebrew` — when accident happened in an intersection, shows street2 that intersects with street1.
4. `location_accuracy_hebrew` — shows how accurate the location is.
5. `accident_type_hebrew` — what or how it happened (fire, cotrol loss, frontal \ side crash, animals, etc.).
6. `accident_severity_hebrew` — if any injured person in the accident is light, serious, lethal.
7. `injury_severity_hebrew` — if the injury of the specific person light, serious, lethal.
8. `injured_type_modified_by_anyway` — modified column that combines the following two columns.
9. `injured_type_hebrew` — who was injured - pedestrian, driver, passenger.
10. `involve_vehicle_type_hebrew` — what vehicle is involved.
11. `involved_type_hebrew` —  is the person a driver or not?.
12. `injured_position_hebrew` — what is the position of the injred person (unknown, pedestrian, sat, stand etc.).
13. `sex_hebrew` — gender.
14. `age_group_hebrew` — age group of 5 years (25-29, 30-34, etc.).
15. `accident_day` — day of the month (1-31).
16. `accident_month` — month number (1-12).
17. `accident_year` — year (2018-2023).
18. `accident_hour_raw_hebrew` — time of accident in 15 minutes intervals (2:00-2:14, 2:15-2:29, etc.).
19. `day_in_week_hebrew` — day of the week (sunday - saturday).

In [4]:
tlv_accidents.sample(3)

,provider_and_id,street1_hebrew,street2_hebrew,location_accuracy_hebrew,accident_type_hebrew,accident_severity_hebrew,injury_severity_hebrew,injured_type_modified_by_anyway,injured_type_hebrew,involve_vehicle_type_hebrew,involved_type_hebrew,injured_position_hebrew,sex_hebrew,age_group_hebrew,accident_day,accident_month,accident_year,accident_hour_raw_hebrew,day_in_week_hebrew
8006,32019056779,יפת,NaN,עיגון מדויק,התנגשות צד בצד,קלה,פצוע קל,"אופנוע עד 50 סמ""ק",נהג - אופנוע,"אופנוע עד 50 סמ""ק",נהג נפגע,ישב ברכב במושב קדמי,נקבה,15-19,1,11,2019,17:45-17:59,שישי
4986,32018088731,דרך בגין,NaN,עיגון מדויק,התנגשות חזית באחור,קלה,פצוע קל,רכב נוסעים פרטי,נוסע - רכב בעל 4 גלגלים ויותר,רכב נוסעים פרטי,נפגע,לא ידוע,נקבה,50-54,27,3,2018,10:30-10:44,שלישי
12663,12020065995,גר צדק,NaN,עיגון מדויק,התנגשות צד בצד,קלה,פצוע קל,מונית,נהג - רכב בעל 4 גלגלים ויותר,מונית,נהג נפגע,ישב ברכב במושב קדמי,זכר,65-69,23,9,2020,12:30-12:44,רביעי


## Preprocessing
The plan:
- `timestamp` column will be created from the four columns: `accident_day`, `accident_month`, `accident_year` and `accident_hour_raw_hebrew`.    
    
    
- The "מרכז ישוב" value from the `location_accuracy_hebrew` column is not accurate enough and will be removed.


- Delete the following columns
    - `involve_vehicle_type_hebrew`
    - `location_accuracy_hebrew`
    - `accident_day`, `accident_month`, `accident_year` and `accident_hour_raw_hebrew`


- `injured_type_modified_by_anyway` column will be renamed to `involved`.


- Creating a list of intersections where accidents happened by combining the values in `street1_hebrew` and `street2_hebrew` columns (in this form: "street1 | street2").


- Creating a list of streets with accidents from both `street1_hebrew` and `street2_hebrew` columns.
    - **IMPORTANT NOTE! — if an injury appears to happen in an intersection it will show up twice, once per intersecting street of the same intersection.**

### Creating `timestamp` column

In [5]:
# creating a date time tlv_accidents type from all time columns (day - month - year - hour) -- keeping the first quarter hour
tlv_accidents['timestamp'] = pd.to_datetime(
    tlv_accidents['accident_day'].astype(str) + '-' +
    tlv_accidents['accident_month'].astype(str) + '-' +
    tlv_accidents['accident_year'].astype(str) + ' ' +
    tlv_accidents['accident_hour_raw_hebrew'].str.split('-').str[0], #split and keep first part of the split only
    format='%d-%m-%Y %H:%M')

#converting string to date type and making the format a little easier to read
tlv_accidents['timestamp'] = pd.to_datetime(tlv_accidents['timestamp'].dt.strftime('%d-%m-%Y %H:%M'))

print('tlv_accidentstype of the timestamp column: {}'.format(tlv_accidents['timestamp'].dtype))
display(tlv_accidents[['timestamp', 'accident_day', 'accident_month', 'accident_year', 'accident_hour_raw_hebrew']].sample(5))

#comment: it is not possible to change the format to "d-m-y H:M" without changing tlv_accidents type back to object

tlv_accidentstype of the timestamp column: datetime64[ns]


C:\Users\chezi\AppData\Local\Temp\ipykernel_12836\3700438600.py:10: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  tlv_accidents['timestamp'] = pd.to_datetime(tlv_accidents['timestamp'].dt.strftime('%d-%m-%Y %H:%M'))


,timestamp,accident_day,accident_month,accident_year,accident_hour_raw_hebrew
5292,2018-05-11 10:15:00,11,5,2018,10:15-10:29
3678,2018-02-06 08:15:00,6,2,2018,08:15-08:29
9830,2019-05-27 08:15:00,27,5,2019,08:15-08:29
7739,2019-05-22 13:30:00,22,5,2019,13:30-13:44
24241,2023-05-25 19:30:00,25,5,2023,19:30-19:44


### Removing any rows with the value of "מרכז ישוב" from the `location_accuracy_hebrew` column
- These locations are not accurate enough

In [6]:
tlv_accidents = tlv_accidents[tlv_accidents['location_accuracy_hebrew']!='מרכז ישוב']

### Removing unnecessary columns:
- `involve_vehicle_type_hebrew`
    - Since the new `involved` column (previously `injured_type_modified_by_anyway`) has the same and more values.


- `location_accuracy_hebrew`
   -  We do not need to know if the location is accurate or not anymore. 
   
   
- These columns are united into one date time column and should also be removed:
    - `accident_day	`
    - `accident_month`
    - `accident_year`
    - `accident_hour_raw_hebrew`
    

In [7]:
tlv_accidents = tlv_accidents.drop(['involve_vehicle_type_hebrew',
                                    'location_accuracy_hebrew',
                                    'accident_day',
                                    'accident_month',
                                    'accident_year',
                                    'accident_hour_raw_hebrew'
                                   ], axis=1)

### Renaming the `injured_type_modified_by_anyway` to `involved`

In [8]:
tlv_accidents = tlv_accidents.rename(columns = {'injured_type_modified_by_anyway': 'involved'})

### Re-categorizing the `involved` column categories

#### original categories and numbers:

In [9]:
tlv_accidents['involved'].value_counts().sort_index().to_frame()

,count
involved,
אוטובוס,1980
אוטובוס זעיר,93
"אופנוע 126 עד 400 סמ""ק",2437
"אופנוע 401+ סמ""ק",919
"אופנוע 51 עד 125 סמ""ק",3086
"אופנוע עד 50 סמ""ק",255
אופניים,855
אופניים חשמליים,1558
אחר ולא ידוע,485


#### We will change and combine the categories as follows:

- **אוטובוס ואוטובוס זעיר**
    - Combined categories of:
        - אוטובוס
        - אוטובוס זעיר 


- **אופנוע**
    - Combined categories of:
        - אופנוע עד 50 סמ"ק
        - אופנוע 51 עד 125 סמ"ק
        - אופנוע 126 עד 400 סמ"ק
        - אופנוע 401+ סמ"ק
        
        
- **אופניים**
    - No change.


- **אופניים חשמליים**
    - No change.


- **אחר ולא ידוע**
    - Combined categories of:
        - אחר ולא ידוע
        - קלנועית חשמלית
        - רכבת


- **הולך רגל**
    - No change.


- **קורקינט חשמלי**
    - No change.


- **רכב כבד**
    - Combined categories of:
        - משא עד 3.5 טון - אחוד (טרנזיט)
        - משא עד 3.5  טון - לא אחוד (טנדר)
        - משא 3.6 עד 9.9 טון
        - משא 10.0 עד 12.0 טון
        - משא 12.1 עד 15.9 טון
        - משא 16.0 עד 33.9 טון
        - משא 34.0+ טון
        - טרקטור
        - רכב עבודה


- **רכב נוסעים פרטי ומונית**
    - Combined categories of:
        - רכב נוסעים פרטי
        - מונית



**important to note:** numbers are after pre-processing the data

#### Recategorization of the column

In [10]:
'''change string values of categories in the involved column using regex'''
tlv_accidents['involved'] = ( 
    tlv_accidents['involved']
        .str.replace('.*אופנוע.*', 'אופנוע', regex=True) # איפה שמופיעה המילה אופנוע בין כל המילים, מחליף ל"אופנוע" בלבד
        .str.replace('.*אוטובוס.*', 'אוטובוס ואוטובוס זעיר', regex=True) # איפה שמופיע "אוטובוס" בין כל המילים, מחליף ל"אוטובוס ואוטובוס זעיר" בלבד
        .str.replace('מונית|רכב נוסעים פרטי', 'רכב נוסעים פרטי ומונית', regex=True) # מחליף "מונית" ו"רכב נוסעים פרטי" ל"רכב נוסעים פרטי ומונית" בלבד
        .str.replace('.*משא.*|טרקטור|רכב עבודה', 'רכב כבד', regex=True) # איפה שמופיע "משא", "טרקטור", "רכב עבודה" מחליף ל"רכב כבד"
        .str.replace('קלנועית חשמלית|רכבת', 'אחר ולא ידוע', regex=True) # מחליף "קלנועית חשמלית" ו"רכבת" ומאחד עם "אחר ולא ידוע" שכבר קיים
)

In [11]:
tlv_accidents['involved'].value_counts().sort_index().to_frame()

,count
involved,
אוטובוס ואוטובוס זעיר,2073
אופנוע,6697
אופניים,855
אופניים חשמליים,1558
אחר ולא ידוע,526
הולך רגל,3016
קורקינט חשמלי,1330
רכב כבד,433
רכב נוסעים פרטי ומונית,7694


## Creating a list of intersections from the names of both streets where the accident happened
This will be done with a function that takes `street1_hebrew` name, and combines it with `street2_hebrew` name. The function will make it possible to have only one combination for each pair of street names, where the alphabetically lower street name comes first as street1.
- `street1_hebrew` and `street2_hebrew` columns contain the street names where the accident happened.
- If the column `street2_hebrew` contains a value, it means the accident took place in a intersections where street1 and street2 meet
    - Otherwise, if there's no value in `street2_hebrew`, the accident happened in the middle of `street1_hebrew`, and not on a intersection.

In [12]:
#defining all intersections (where `street2_hebrew` is not null) creating a copy from the original data table
tlv_intersection_injuries = tlv_accidents[tlv_accidents['street2_hebrew'].notnull()].copy()

'''
    function that takes two variables (from street 1 and 2),
    and combines them to forn the intersection name ("street1 | street2")
    in such way that the first street name will be alphabetically lower than the second street name
'''
def intersection_name(str1, str2):
    if str1 < str2:  #if the name of str1 is smaller than (alphabetically comes before) str2
        return str1 +" | "+ str2 #first take the string from str1 and combine with " | " AND "str2"
    else: #if str1 NOT smaller (alphabetically comes before) str2
        return str2 + " | " + str1  #combine first "str2" + " | " + "str1"
    
    
#applying the function to combine street names into intersection
tlv_intersection_injuries['intersection'] = tlv_intersection_injuries.apply( 
                            lambda x: intersection_name(x['street1_hebrew'], x['street2_hebrew']), axis=1)

In [13]:
tlv_intersection_injuries.sample(3)

,provider_and_id,street1_hebrew,street2_hebrew,accident_type_hebrew,accident_severity_hebrew,injury_severity_hebrew,involved,injured_type_hebrew,involved_type_hebrew,injured_position_hebrew,sex_hebrew,age_group_hebrew,day_in_week_hebrew,timestamp,intersection
24230,32023036779,יהודה הלוי,שיינקין,התנגשות חזית בצד,קלה,פצוע קל,אופנוע,נהג - אופנוע,נהג נפגע,ישב ברכב במושב קדמי,זכר,55-59,רביעי,2023-07-12 04:00:00,יהודה הלוי | שיינקין
6182,32019029495,המלך ג'ורג',אלנבי,התנגשות צד בצד,קלה,פצוע קל,אופנוע,נהג - אופנוע,נהג נפגע,ישב ברכב במושב קדמי,זכר,30-34,שני,2019-07-22 18:00:00,אלנבי | המלך ג'ורג'
3368,32018064556,פרישמן,שלמה המלך,התנגשות חזית באחור,קלה,פצוע קל,אופנוע,נהג - אופנוע,נהג נפגע,ישב ברכב במושב קדמי,זכר,35-39,ראשון,2018-06-24 12:00:00,פרישמן | שלמה המלך


#### Saving injuries by intersection table as csv

In [14]:
#tlv_intersection_injuries.to_csv('tlv_intersection_injuries.csv', index=False)

## Creating a list of streets where accidents happened
- We intend to keep all streets intact from both `street1_hebrew` and `street2_hebrew` columns, as it is most logical; when accident takes place in an intersection between two streets, both streets count as the location of the accident.
    - **IMPORTANT NOTE — This also means that the same accident data will show up twice, once per street (if accident occured in an intersection)**
    - In other words: if we count all accidents from this data, the numbers will be greater than actual accidents because the same accident may have two rows of data - one per street in an intersection.
- This list will be done by concatenating all values from the original data with the samae data for where `street2_hebrew` column has values, then combining both columns into one : `street`.

In [15]:
#getting all data where `street2_hebrew` has values (non-null):
tlv_2nd_street = tlv_accidents[~tlv_accidents['street2_hebrew'].isnull()].copy() 

#renaming the values in `street1_hebrew` to be the same as `street2_hebrew`
tlv_2nd_street['street1_hebrew'] = tlv_2nd_street['street2_hebrew']

#concatenating the data from the previous table and the original data
tlv_street_injuries = pd.concat([tlv_accidents, tlv_2nd_street], axis = 0, ignore_index=True,)

#removing the `stree2_hebrew` column as it's not needed anymore
tlv_street_injuries.drop('street2_hebrew', axis = 1, inplace = True)

#renaming `street1_hebrew` to `street`
tlv_street_injuries.rename(columns = {"street1_hebrew":"street"}, inplace = True)
tlv_street_injuries.sample(3)

,provider_and_id,street,accident_type_hebrew,accident_severity_hebrew,injury_severity_hebrew,involved,injured_type_hebrew,involved_type_hebrew,injured_position_hebrew,sex_hebrew,age_group_hebrew,day_in_week_hebrew,timestamp
8063,12019066125,דרך בר-לב חיים,התנגשות חזית בצד,קלה,פצוע קל,רכב נוסעים פרטי ומונית,נהג - רכב בעל 4 גלגלים ויותר,נהג נפגע,ישב ברכב במושב קדמי,זכר,20-24,שני,2019-09-02 08:15:00
2497,12018046158,סומקן,התנגשות חזית בצד,קלה,פצוע קל,רכב נוסעים פרטי ומונית,נוסע - רכב בעל 4 גלגלים ויותר,נפגע,לא ידוע,נקבה,20-24,שישי,2018-03-16 18:00:00
12143,32020047835,קפלן,אחר,קלה,פצוע קל,אופניים,נהג - אופניים,נהג נפגע,ישב ברכב במושב קדמי,זכר,30-34,שלישי,2020-02-04 13:00:00


#### Saving injuries by sreet table as csv

In [16]:
#tlv_street_injuries.to_csv('tlv_street_injuries.csv', index=False)

## Creating a table to sum injuries by severity and involved category —  per intersection

### Creating 3 pivot tables
- The tables will sum each involved category (vehicle or pedestrian) as a column, per injury severity (light, serious, or fatal).
- To be combined into one large table.

In [17]:
'''---slicing only injured people with light injury (independed on accident severity)---'''

intersection_light_inj = tlv_intersection_injuries[tlv_intersection_injuries['injury_severity_hebrew'] == "פצוע קל"]

#pivoting light injuries per vehicle type
intersection_light_inj_pivot = (
    intersection_light_inj.pivot_table(
        index = 'intersection', 
        columns = 'involved', 
        values = 'injury_severity_hebrew', 
        aggfunc = 'count', 
        fill_value = 0 # replacing missing values (NaN) cause by aggregation with 0
    )
    .reset_index()
)

# removing column name that appears above the index
intersection_light_inj_pivot.columns.name = None 

# renaming columns with קשה
intersection_light_inj_pivot.columns = (
    ['צומת'] + [col + '_קל' for col in intersection_light_inj_pivot.columns[1:]])

# adding sum column for all light injuries
intersection_light_inj_pivot["סך_פצועים_קל"] = (
    intersection_light_inj_pivot[intersection_light_inj_pivot.columns[1:]].sum(axis = 1))

print('sample light injuries pivot:')
display(intersection_light_inj_pivot.head(3))



'''---slicing only injured people with serious injury (independed on accident severity)---'''

intersection_severe_inj = tlv_intersection_injuries[tlv_intersection_injuries['injury_severity_hebrew'] == "פצוע קשה"]

#pivoting serious injuries per vehicle type
intersection_severe_inj_pivot = (
    intersection_severe_inj.pivot_table(
        index = 'intersection', 
        columns = 'involved', 
        values = 'injury_severity_hebrew', 
        aggfunc = 'count', 
        fill_value = 0 # replacing missing values (NaN) cause by aggregation with 0
    )
    .reset_index()
)

# removing column name that appears above the index
intersection_severe_inj_pivot.columns.name = None 

#renaming columns with קשה
intersection_severe_inj_pivot.columns = (
    ['צומת'] + [col + '_קשה' for col in intersection_severe_inj_pivot.columns[1:]])

# adding sum column for all serious injuries
intersection_severe_inj_pivot["סך_פצועים_קשה"] = (
    intersection_severe_inj_pivot[intersection_severe_inj_pivot.columns[1:]].sum(axis = 1))


print('\n\nsample serious injuries pivot:')
display(intersection_severe_inj_pivot.head(3))



'''---slicing only deaths (independed on accident severity)---'''

intersection_fatal_inj = tlv_intersection_injuries[tlv_intersection_injuries['injury_severity_hebrew'] == "הרוג"]

#pivoting the fatalisties to show how many deaths per vehicle type
intersection_fatal_inj_pivot = (
    intersection_fatal_inj.pivot_table(
        index = 'intersection', 
        columns = 'involved', 
        values = 'injury_severity_hebrew', 
        aggfunc = 'count', 
        fill_value = 0 # replacing missing values (NaN) cause by aggregation with 0
    )
    .reset_index()
)


# removing column name that appears above the index
intersection_fatal_inj_pivot.columns.name = None 

#renaming columns with הרוגים
intersection_fatal_inj_pivot.columns = (
    ['צומת'] + [col + '_הרוגים' for col in intersection_fatal_inj_pivot.columns[1:]]) 

# adding sum column for all fatal injuries
intersection_fatal_inj_pivot['סך_הרוגים'] = (
    intersection_fatal_inj_pivot[intersection_fatal_inj_pivot.columns[1:]].sum(axis = 1))


print('sample fatal pivot:\n')
display(intersection_fatal_inj_pivot.head(3))


sample light injuries pivot:


,צומת,אוטובוס ואוטובוס זעיר_קל,אופנוע_קל,אופניים_קל,אופניים חשמליים_קל,אחר ולא ידוע_קל,הולך רגל_קל,קורקינט חשמלי_קל,רכב כבד_קל,רכב נוסעים פרטי ומונית_קל,סך_פצועים_קל
0,אבולעפיה | דרך שלמה,0,1,0,0,0,2,0,0,0,3
1,"אבולעפיה | הש""ך",0,0,0,0,0,0,0,0,1,1
2,אבולעפיה | קבוץ גלויות,0,0,0,1,0,0,0,0,3,4




sample serious injuries pivot:


,צומת,אוטובוס ואוטובוס זעיר_קשה,אופנוע_קשה,אופניים_קשה,אופניים חשמליים_קשה,אחר ולא ידוע_קשה,הולך רגל_קשה,קורקינט חשמלי_קשה,רכב נוסעים פרטי ומונית_קשה,סך_פצועים_קשה
0,אבן ג'נאח | אבן גבירול,1,0,0,0,0,0,0,0,1
1,אבן גבירול | בזל,0,0,1,0,0,0,0,0,1
2,אבן גבירול | דיזנגוף,0,1,0,0,0,1,0,0,2


sample fatal pivot:



,צומת,אופנוע_הרוגים,אופניים_הרוגים,אופניים חשמליים_הרוגים,אחר ולא ידוע_הרוגים,הולך רגל_הרוגים,קורקינט חשמלי_הרוגים,סך_הרוגים
0,אבן גבירול | ז'בוטינסקי,0,1,0,0,1,0,2
1,"אבן גבירול | עגנון ש""י",0,0,0,0,1,0,1
2,אבן גבירול | שד דוד המלך,0,0,0,0,1,0,1


### Merging the 3 injury severity and involved categories tables

In [18]:
inter_kalkashe = pd.merge(# merge first two tables)
    how = 'outer', 
    on = 'צומת', 
    left = intersection_light_inj_pivot, 
    right = intersection_severe_inj_pivot)


inter_all_severities_involved = pd.merge(# merge the first merged tables with the 3rd one
    how = 'outer', 
    on = 'צומת', 
    left = inter_kalkashe, 
    right = intersection_fatal_inj_pivot)

# fill null values created by the merging with 0
inter_all_severities_involved = inter_all_severities_involved.fillna(0) 

# change float values created by the missing values when merging, skipping first column
inter_all_severities_involved.iloc[:,1:] = inter_all_severities_involved.iloc[:,1:].astype('int')

#adding a sum column to sum all sum columns for each injury level
inter_all_severities_involved['כל_הנפגעים'] = (
    inter_all_severities_involved["סך_פצועים_קל"] + 
    inter_all_severities_involved["סך_פצועים_קשה"] + 
    inter_all_severities_involved["סך_הרוגים"])


inter_all_severities_involved.sort_values('צומת', inplace = True)# sort alphabetically
inter_all_severities_involved.head()

,צומת,אוטובוס ואוטובוס זעיר_קל,אופנוע_קל,אופניים_קל,אופניים חשמליים_קל,אחר ולא ידוע_קל,הולך רגל_קל,קורקינט חשמלי_קל,רכב כבד_קל,רכב נוסעים פרטי ומונית_קל,...,רכב נוסעים פרטי ומונית_קשה,סך_פצועים_קשה,אופנוע_הרוגים,אופניים_הרוגים,אופניים חשמליים_הרוגים,אחר ולא ידוע_הרוגים,הולך רגל_הרוגים,קורקינט חשמלי_הרוגים,סך_הרוגים,כל_הנפגעים
0,אבולעפיה | דרך שלמה,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
1,"אבולעפיה | הש""ך",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,אבולעפיה | קבוץ גלויות,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
3,אבולעפיה | שד ושינגטון,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,"אבוקה | צה""ל",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [19]:
inter_all_severities_involved.shape

(1457, 28)

In [20]:
inter_all_severities_involved.duplicated(subset='צומת').sum()

0

### Creaeting an Excel file with the intersection injuries per injury severity and involved category

In [21]:
inter_all_severities_involved.to_excel('tlv_intersection_injuries_involved_sum.xlsx', index=False)  

## Creating a table that sums injuries by severity and involved category  — per street
Notice the sum of the injuries in total is larger than is, this is because injuries that happened in an intersections are listed twice - once on each intersecting street

### Creating 3 pivot tables
- The tables will sum each involved category (vehicle or pedestrian) as a column, per injury severity (light, serious, or fatal).
- To be combined into one large table.

In [22]:
'''---slicing only injured people with light injury (independed on accident severity)---'''

street_light_inj = tlv_street_injuries[tlv_street_injuries['injury_severity_hebrew'] == "פצוע קל"]

#pivoting light injuries per vehicle type
street_light_inj_pivot = (
    street_light_inj.pivot_table(
        index = 'street', 
        columns = 'involved', 
        values = 'injury_severity_hebrew', 
        aggfunc = 'count', 
        fill_value = 0 # replacing missing values (NaN) caused by aggregation with 0
    )
    .reset_index()
)

# removing column name that appears above the index
street_light_inj_pivot.columns.name = None

# renaming column to add _קל to it 
street_light_inj_pivot.columns = (
    ['רחוב'] + [col + '_קל' for col in street_light_inj_pivot.columns[1:]])

# creating a sum column for all light injuries
street_light_inj_pivot["סך_פצועים_קל"] = (
    street_light_inj_pivot[street_light_inj_pivot.columns[1:]].sum(axis = 1))


print('sample light injuries pivot:')
display(street_light_inj_pivot.head(3))



'''---slicing only injured people with serious injury (independed on accident severity)---'''

street_severe_inj = tlv_street_injuries[tlv_street_injuries['injury_severity_hebrew'] == "פצוע קשה"]

#pivoting serious injuries per vehicle type
street_severe_inj_pivot = (
    street_severe_inj.pivot_table(
        index = 'street', 
        columns = 'involved', 
        values = 'injury_severity_hebrew', 
        aggfunc = 'count', 
        fill_value = 0 # replacing missing values (NaN) caused by aggregation with 0
    )
    .reset_index()
)

# removing column name that appears above the index
street_severe_inj_pivot.columns.name = None 

# renaming columns with קשה
street_severe_inj_pivot.columns = (
    ['רחוב'] + [col + '_קשה' for col in street_severe_inj_pivot.columns[1:]]) 

# creating a sum column for all serious injuries
street_severe_inj_pivot["סך_פצועים_קשה"] = (
    street_severe_inj_pivot[street_severe_inj_pivot.columns[1:]].sum(axis = 1))


print('\n\nsample serious injuries pivot:')
display(street_severe_inj_pivot.head(3))



'''---slicing only deaths (independed on accident severity)---'''

street_fatal_inj = tlv_street_injuries[tlv_street_injuries['injury_severity_hebrew'] == "הרוג"]

#pivoting the fatalisties to show how many deaths per vehicle type
street_fatal_inj_pivot = (
    street_fatal_inj.pivot_table(
        index = 'street', 
        columns = 'involved', 
        values = 'injury_severity_hebrew', 
        aggfunc = 'count', 
        fill_value = 0 # replacing missing values (NaN) caused by aggregation with 0
    )
    .reset_index()
)

# removing column name that appears above the index
street_fatal_inj_pivot.columns.name = None 

# renaming columns with הרוגים
street_fatal_inj_pivot.columns = (
    ['רחוב'] + [col + '_הרוגים' for col in street_fatal_inj_pivot.columns[1:]]) 


# creating a sum column for all fatal injuries
street_fatal_inj_pivot['סך_הרוגים'] = street_fatal_inj_pivot[street_fatal_inj_pivot.columns[1:]].sum(axis = 1)


print('sample fatal pivot:\n')
display(street_fatal_inj_pivot.head(3))


sample light injuries pivot:


,רחוב,אוטובוס ואוטובוס זעיר_קל,אופנוע_קל,אופניים_קל,אופניים חשמליים_קל,אחר ולא ידוע_קל,הולך רגל_קל,קורקינט חשמלי_קל,רכב כבד_קל,רכב נוסעים פרטי ומונית_קל,סך_פצועים_קל
0,אב,0,0,0,0,1,0,1,0,0,2
1,אבולעפיה,0,2,1,1,0,2,1,0,7,14
2,אבוקה,0,0,0,1,0,0,0,0,2,3




sample serious injuries pivot:


,רחוב,אוטובוס ואוטובוס זעיר_קשה,אופנוע_קשה,אופניים_קשה,אופניים חשמליים_קשה,אחר ולא ידוע_קשה,הולך רגל_קשה,קורקינט חשמלי_קשה,רכב כבד_קשה,רכב נוסעים פרטי ומונית_קשה,סך_פצועים_קשה
0,אבן ג'נאח,1,0,0,0,0,0,0,0,0,1
1,אבן גבירול,8,8,3,1,1,8,3,0,1,33
2,אבן שושן אברהם,0,1,0,0,0,0,0,0,0,1


sample fatal pivot:



,רחוב,אופנוע_הרוגים,אופניים_הרוגים,אופניים חשמליים_הרוגים,אחר ולא ידוע_הרוגים,הולך רגל_הרוגים,קורקינט חשמלי_הרוגים,רכב נוסעים פרטי ומונית_הרוגים,סך_הרוגים
0,אבן גבירול,0,1,0,0,3,0,0,4
1,אינשטיין,1,0,0,1,0,0,0,2
2,אלון יגאל,0,0,1,0,2,1,0,4


In [23]:
# merge the first two tables
streets_kalkashe = pd.merge(
    how = 'outer', 
    on = 'רחוב', 
    left = street_light_inj_pivot, 
    right = street_severe_inj_pivot)

# merge the first merged tables with the 3rd one
streets_all_severities_involved = pd.merge(
    how = 'outer', 
    on = 'רחוב', 
    left = streets_kalkashe, 
    right = street_fatal_inj_pivot)

# fill null values created by the merging with 0
streets_all_severities_involved = streets_all_severities_involved.fillna(0)

# change float values created by the missing values when merging, skipping first column
streets_all_severities_involved.iloc[:,1:] = streets_all_severities_involved.iloc[:,1:].astype('int')

#adding a sum column to sum all sum columns for each injury level
streets_all_severities_involved['כל_הנפגעים'] = (
    streets_all_severities_involved["סך_פצועים_קל"] + 
    streets_all_severities_involved["סך_פצועים_קשה"] + 
    streets_all_severities_involved["סך_הרוגים"])


streets_all_severities_involved.sort_values('רחוב', inplace = True) # sort alphabetically
streets_all_severities_involved.head()

,רחוב,אוטובוס ואוטובוס זעיר_קל,אופנוע_קל,אופניים_קל,אופניים חשמליים_קל,אחר ולא ידוע_קל,הולך רגל_קל,קורקינט חשמלי_קל,רכב כבד_קל,רכב נוסעים פרטי ומונית_קל,...,סך_פצועים_קשה,אופנוע_הרוגים,אופניים_הרוגים,אופניים חשמליים_הרוגים,אחר ולא ידוע_הרוגים,הולך רגל_הרוגים,קורקינט חשמלי_הרוגים,רכב נוסעים פרטי ומונית_הרוגים,סך_הרוגים,כל_הנפגעים
0,אב,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,אבולעפיה,0.0,2.0,1.0,1.0,0.0,2.0,1.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0
2,אבוקה,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
3,אביאסף,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,אבינרי יצחק,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [24]:
streets_all_severities_involved.shape

(1312, 30)

In [25]:
streets_all_severities_involved.duplicated(subset='רחוב').sum()

0

### Creating an Excel file with the intersection injuries per injury severity and involved category

In [26]:
streets_all_severities_involved.to_excel('tlv_street_injuries_involved_sum.xlsx', index=False)  